<a href="https://colab.research.google.com/github/3zero69/-/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 물류 유통량 예측 경진대회

https://dacon.io/competitions/official/235867/overview/description

* 목적
: 제주시 내 택배 운송 데이터를 이용하여 운송량 예측 AI 개발

* 데이터 정보
  - train_df.csv

    - index : 인덱스
    - 송하인_격자공간고유번호
    - 수하인 격자공간고유번호
    - 물품_카테고리
    - 운송장_건수
  - test_df.csv
    - index : 인덱스
    - 송하인_격자공간고유번호
    - 수하인 격자공간고유번호
    - 물품_카테고리
  - sample_submission.csv
    - index : 인덱스
    - 운송장_건수




## 데이터 불러오기  및 기본 info()

In [ ]:
#import pandas as pd
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

file_path = '/content/gdrive/My Drive/Colab Notebooks/ESAA_OB/project1/'


Mounted at /content/gdrive/


In [ ]:
# 파일 불러오기 

train = pd.read_csv(file_path + 'train.csv')
test = pd.read_csv(file_path + 'test.csv')

submission = pd.read_csv(file_path + 'sample_submission.csv')


#### 데이터 기본정보

In [ ]:
train.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3
3,3,4127100048006400,5011000587019400,기타식품,7
4,4,5011000078068400,2823700010076300,농산물,3


In [ ]:
test.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
0,0,4167000577042200,5011000435014100,선케어
1,1,1156000009012200,5011000172034400,구강위생용품
2,2,4122000363057300,5011000361097300,캠핑
3,3,5011000436041400,2826000084036400,아웃도어가구
4,4,4150000241065200,5011000169044300,분유/이유식/아기간식


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         31684 non-null  int64 
 1   송하인_격자공간고유번호  31684 non-null  int64 
 2   수하인_격자공간고유번호  31684 non-null  int64 
 3   물품_카테고리       31684 non-null  object
 4   운송장_건수        31684 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1.2+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         7920 non-null   int64 
 1   송하인_격자공간고유번호  7920 non-null   int64 
 2   수하인_격자공간고유번호  7920 non-null   int64 
 3   물품_카테고리       7920 non-null   object
dtypes: int64(3), object(1)
memory usage: 247.6+ KB


In [ ]:
train.isnull().sum()

index           0
송하인_격자공간고유번호    0
수하인_격자공간고유번호    0
물품_카테고리         0
운송장_건수          0
dtype: int64

In [ ]:
train.columns

Index(['index', '송하인_격자공간고유번호', '수하인_격자공간고유번호', '물품_카테고리', '운송장_건수'], dtype='object')

## 데이터 전처리

### '물품 카테고리' 칼럼

In [ ]:
train['물품_카테고리'].value_counts(dropna=False)

농산물              20321
문화컨텐츠             1091
음료                1034
수산                 860
가공식품               846
                 ...  
유아가구                 4
기타출산/육아              4
태블릿PC/노트북액세서리        4
선케어                  4
스포츠잡화                4
Name: 물품_카테고리, Length: 100, dtype: int64

In [ ]:
train['물품_카테고리'].unique()

array(['음반', '문화컨텐츠', '농산물', '기타식품', '수산', '아우터', '신발', '건강식품', '음료',
       '가공식품', '스킨케어', '축산', '헤어케어', '기저귀/물티슈', '의료용품', '기타디지털/가전', '상의',
       '건강용품', '바디케어', '기타스포츠/레저', '출산/유아동의류', '반려동물', '완구/매트', '과자',
       '생활용품', '하의', '문구/사무용품', '선케어', '다이어트식품', '기타패션의류', '냉동/간편조리식품',
       '네일케어', '서재/사무용가구', '선글라스/안경테', '기타화장품/미용', '등산', '모니터', '주방용품',
       '색조메이크업', '침구세트', '침구단품', '인테리어소품', '양말/스타킹', '클렌징', '공구', '이미용가전',
       'PC', '커튼/블라인드', '헤어스타일링', '주방가구', 'DIY자재/용품', '헤어액세서리', '김치',
       '위생/건강용품', '침실가구', '가방', '주방가전', '낚시', '재활운동용품', '자동차용품', '뷰티소품',
       '주얼리', '골프', 'PC주변기기', '수납/정리용품', '음향가전', '잠옷', '패션소품', '언더웨어',
       '원피스/점프슈트', '스포츠잡화', '수납가구', '취미용품', '아웃도어가구', '계절가전', '홈데코', '캠핑',
       '남성화장품', '분유/이유식/아기간식', '베이스메이크업', '헬스', '향수', '모자', '카페트/러그',
       '구강위생용품', '눈관리용품', '기타출산/육아', '기타패션잡화', '기능성', '반찬', '욕실용품',
       '스마트디바이스액세서리', '게임기/타이틀', '생활가전', '유아가구', '출산/유아동잡화', '세탁용품',
       '태블릿PC/노트북액세서리', '스마트디바이스', '지갑'], dtype=object)

In [ ]:
arr = train['물품_카테고리'].unique()
arr1 = np.array2string(arr)
str(arr1)

arr1.replace("'", " ")

'[ 음반   문화컨텐츠   농산물   기타식품   수산   아우터   신발   건강식품   음료   가공식품   스킨케어   축산 \n  헤어케어   기저귀/물티슈   의료용품   기타디지털/가전   상의   건강용품   바디케어   기타스포츠/레저 \n  출산/유아동의류   반려동물   완구/매트   과자   생활용품   하의   문구/사무용품   선케어   다이어트식품 \n  기타패션의류   냉동/간편조리식품   네일케어   서재/사무용가구   선글라스/안경테   기타화장품/미용   등산   모니터 \n  주방용품   색조메이크업   침구세트   침구단품   인테리어소품   양말/스타킹   클렌징   공구   이미용가전   PC \n  커튼/블라인드   헤어스타일링   주방가구   DIY자재/용품   헤어액세서리   김치   위생/건강용품   침실가구   가방 \n  주방가전   낚시   재활운동용품   자동차용품   뷰티소품   주얼리   골프   PC주변기기   수납/정리용품   음향가전 \n  잠옷   패션소품   언더웨어   원피스/점프슈트   스포츠잡화   수납가구   취미용품   아웃도어가구   계절가전   홈데코 \n  캠핑   남성화장품   분유/이유식/아기간식   베이스메이크업   헬스   향수   모자   카페트/러그   구강위생용품 \n  눈관리용품   기타출산/육아   기타패션잡화   기능성   반찬   욕실용품   스마트디바이스액세서리   게임기/타이틀 \n  생활가전   유아가구   출산/유아동잡화   세탁용품   태블릿PC/노트북액세서리   스마트디바이스   지갑 ]'

#### '물품_카테고리'

In [ ]:
식음료품 = ['농산물', '음료', '수산', '가공식품', '기타식품', '건강식품',
        '냉동/간편조리식품', '과자', '다이어트식품', '김치', '분유/이유식/아기간식', '반찬']
개인관리용품 = ['스킨케어', '기타화장품/미용', '헤어케어', '베이스메이크업',
          '바디케어', '색조메이크업', '네일케어', '뷰티소품', '눈관리용품', '헤어액세서리',
          '향수', '남성화장품', '헤어스타일링', '클렌징', '선케어']
의류가방 = ['상의', '기타패션의류', '아우터', '하의', '신발', '언더웨어', '원피스/점프슈트', 
         '잠옷', '양말/스타킹', '패션소품', '기타패션잡화',  '출산/유아동의류', '가방', 
         '모자', '재활운동용품', '선글라스/안경테', '지갑','기저귀/물티슈']
산동식물및생산품 = ['축산','반려동물']
가정용품및가전제품 = ['주방용품', '기타디지털/가전', '이미용가전', '주방가전', '음향가전', '계절가전',
             '생활가전', '카페트/러그', '침구단품', '수납/정리용품', '침구세트', '커튼/블라인드',
             '홈데코', '출산/유아동잡화', '기타출산/육아', '생활용품']
정보기술방송및통신기 = ['스마트디바이스', '스마트디바이스액세서리', '모니터', 'PC주변기기', '태블릿PC/노트북액세서리', '게임기/타이틀', 'PC']
가구및관련제품 = ['주방가구', '수납가구', '서재/사무용가구', '아웃도어가구', '침실가구', '유아가구', '인테리어소품', 'DIY자재/용품']
보건서비스 = ['헬스']
스포츠및레크레이션 = ['기타스포츠/레저', '낚시', '캠핑', '스포츠잡화', '등산', '골프', '건강용품']
사무용기기액세서리 = ['문구/사무용품']
공구및범용기계 =['공구'] 
개인용운송기구및액세서리 = ['자동차용품']
위생장비및용품 = ['위생/건강용품', '욕실용품', '구강위생용품', '세탁용품']
악기게임장난감등 = ['완구/매트', '취미용품']
예술관련서비스 = ['문화컨텐츠', '음반']
의약품 =['의료용품']

In [ ]:
train['물품_카테고리_대분류']=' '

In [ ]:
for i in train['물품_카테고리']:
  if i in 식음료품:
    train.loc[train['물품_카테고리']==i, '물품_카테고리_대분류'] = '식음료품'
  elif i in 개인관리용품:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='개인관리용품'
  elif i in 의류가방:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='의류가방'
  elif i in 산동식물및생산품:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='산동식물및생산품'
  elif i in 가정용품및가전제품:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='가정용품및가전제품'
  elif i in 정보기술방송및통신기:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='정보기술방송및통신기'
  elif i in 가구및관련제품:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='가구및관련제품'
  elif i in 보건서비스:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='보건서비스'
  elif i in 스포츠및레크레이션:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='스포츠및레크레이션'
  elif i in 사무용기기액세서리:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='사무용기기액세서리'
  elif i in 공구및범용기계:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='공구및범용기계'
  elif i in 개인용운송기구및액세서리:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='개인용운송기구및액세서리'
  elif i in 위생장비및용품:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='위생장비및용품'
  elif i in 악기게임장난감등:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='악기게임장난감등'
  elif i in 예술관련서비스:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='예술관련서비스'
  else:
    train.loc[train['물품_카테고리']==i,'물품_카테고리_대분류']='의약품'

In [ ]:
train

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,물품_카테고리_대분류
0,0,5011000595017300,2871000192069300,음반,3,예술관련서비스
1,1,4148000690043300,5011000264024400,문화컨텐츠,3,예술관련서비스
2,2,5011000078068400,1120000007005400,농산물,3,식음료품
3,3,4127100048006400,5011000587019400,기타식품,7,식음료품
4,4,5011000078068400,2823700010076300,농산물,3,식음료품
...,...,...,...,...,...,...
31679,31679,4471000290087200,5011000213073200,스포츠잡화,3,스포츠및레크레이션
31680,31680,1129000014045300,5011000319087100,스마트디바이스,4,정보기술방송및통신기
31681,31681,1129000014045300,5011000263065200,스마트디바이스,6,정보기술방송및통신기
31682,31682,4127300065073100,5011000264061200,지갑,7,의류가방


In [ ]:
train['물품_카테고리_대분류'].value_counts()

식음료품            25209
예술관련서비스          1667
의류가방             1224
가정용품및가전제품        1175
산동식물및생산품          698
개인관리용품            541
스포츠및레크레이션         270
가구및관련제품           227
사무용기기액세서리         167
의약품               109
악기게임장난감등          109
공구및범용기계            95
정보기술방송및통신기         79
위생장비및용품            68
개인용운송기구및액세서리       32
보건서비스              14
Name: 물품_카테고리_대분류, dtype: int64

In [ ]:
df = train.copy()

In [ ]:
df.loc[(df['물품_카테고리']=='농산물') | 
   (df['물품_카테고리']=='기타식품') | 
   (df['물품_카테고리']=='수산')|
   (df['물품_카테고리']=='건강식품')|
   (df['물품_카테고리']=='음료')|
   (df['물품_카테고리']=='가공식품')|
   (df['물품_카테고리']=='축산')|
   (df['물품_카테고리']=='과자')|
   (df['물품_카테고리']=='다이어트식품')|
   (df['물품_카테고리']=='냉동/간편조리식품')|
   (df['물품_카테고리']=='김치')|
   (df['물품_카테고리']=='반찬'), '물품_카테고리'] = '식품'

df.loc[(df['물품_카테고리']=='아우터') | 
   (df['물품_카테고리']=='신발') | 
   (df['물품_카테고리']=='상의')|
   (df['물품_카테고리']=='하의')|
   (df['물품_카테고리']=='양말/스타킹')|
   (df['물품_카테고리']=='기타패션의류')|
   (df['물품_카테고리']=='선글라스/안경테')|
   (df['물품_카테고리']=='가방')|
   (df['물품_카테고리']=='주얼리')|
   (df['물품_카테고리']=='잠옷')|
   (df['물품_카테고리']=='패션소품')|
   (df['물품_카테고리']=='언더웨어')|
   (df['물품_카테고리']=='원피스/점프슈트')|
   (df['물품_카테고리']=='모자')|
   (df['물품_카테고리']=='스포츠잡화')|
   (df['물품_카테고리']=='기타패션잡화')|
   (df['물품_카테고리']=='지갑')|
   (df['물품_카테고리']=='헤어액세서리')|
   (df['물품_카테고리']=='기능성'), '물품_카테고리'] = '패션의류/잡화'

df.loc[(df['물품_카테고리']=='스킨케어') | 
   (df['물품_카테고리']=='헤어케어') | 
   (df['물품_카테고리']=='바디케어')|
   (df['물품_카테고리']=='선케어')|
   (df['물품_카테고리']=='네일케어')|
   (df['물품_카테고리']=='기타화장품/미용')|
   (df['물품_카테고리']=='주얼리')|
   (df['물품_카테고리']=='색조메이크업')|
   (df['물품_카테고리']=='클렌징')|
   (df['물품_카테고리']=='헤어스타일링')|
   (df['물품_카테고리']=='뷰티소품')|
   (df['물품_카테고리']=='남성화장품')|
   (df['물품_카테고리']=='베이스메이크업')|
   (df['물품_카테고리']=='향수'), '물품_카테고리'] = '뷰티'

df.loc[(df['물품_카테고리']=='서재/사무용가구') | 
   (df['물품_카테고리']=='주방가구') | 
   (df['물품_카테고리']=='침실가구')|
   (df['물품_카테고리']=='수납가구')|
   (df['물품_카테고리']=='아웃도어가구')|
   (df['물품_카테고리']=='침구세트')|
   (df['물품_카테고리']=='침구단품')|
   (df['물품_카테고리']=='인테리어소품')|
   (df['물품_카테고리']=='커튼/블라인드')|
   (df['물품_카테고리']=='DIY자재/용품')|
   (df['물품_카테고리']=='홈데코')|
   (df['물품_카테고리']=='카페트/러그'), '물품_카테고리'] = '가구/침구'

df.loc[(df['물품_카테고리']=='기타스포츠/레저') | 
   (df['물품_카테고리']=='등산') | 
   (df['물품_카테고리']=='낚시')|
   (df['물품_카테고리']=='골프')|
   (df['물품_카테고리']=='캠핑')|
   (df['물품_카테고리']=='헬스')|
   (df['물품_카테고리']=='음반')|
   (df['물품_카테고리']=='문화컨텐츠'), '물품_카테고리'] = '스포츠/레저/문화'

df.loc[(df['물품_카테고리']=='모니터') | 
   (df['물품_카테고리']=='PC') | 
   (df['물품_카테고리']=='PC주변기기')|
   (df['물품_카테고리']=='게임기/타이틀')|
   (df['물품_카테고리']=='스마트디바이스액세서리')|
   (df['물품_카테고리']=='태블릿PC/노트북액세서리')|
   (df['물품_카테고리']=='스마트디바이스')|
   (df['물품_카테고리']=='기타디지털/가전')|
   (df['물품_카테고리']=='주방가전')|
   (df['물품_카테고리']=='음향가전')|
   (df['물품_카테고리']=='계절가전')|
   (df['물품_카테고리']=='생활가전')|
   (df['물품_카테고리']=='이미용가전'), '물품_카테고리'] = '가전/디지털'

df.loc[(df['물품_카테고리']=='기저귀/물티슈') | 
   (df['물품_카테고리']=='기타출산/육아') | 
   (df['물품_카테고리']=='분유/이유식/아기간식')|
   (df['물품_카테고리']=='출산/유아동의류')|
   (df['물품_카테고리']=='출산/유아동잡화')|
   (df['물품_카테고리']=='유아가구')|
   (df['물품_카테고리']=='반려동물'), '물품_카테고리'] = '출산/유아동/반려동물'

df.loc[(df['물품_카테고리']=='의료용품') | 
   (df['물품_카테고리']=='건강용품') | 
   (df['물품_카테고리']=='생활용품')|
   (df['물품_카테고리']=='문구/사무용품')|
   (df['물품_카테고리']=='주방용품')|
   (df['물품_카테고리']=='위생/건강용품')|
   (df['물품_카테고리']=='재활운동용품')|
   (df['물품_카테고리']=='자동차용품')|
   (df['물품_카테고리']=='수납/정리용품')|
   (df['물품_카테고리']=='취미용품')|
   (df['물품_카테고리']=='구강위생용품')|
   (df['물품_카테고리']=='눈관리용품')|
   (df['물품_카테고리']=='욕실용품')|
   (df['물품_카테고리']=='세탁용품')|
   (df['물품_카테고리']=='공구')|
   (df['물품_카테고리']=='완구/매트'), '물품_카테고리'] = '생활/건강'

df

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,스포츠/레저/문화,3
1,1,4148000690043300,5011000264024400,스포츠/레저/문화,3
2,2,5011000078068400,1120000007005400,식품,3
3,3,4127100048006400,5011000587019400,식품,7
4,4,5011000078068400,2823700010076300,식품,3
...,...,...,...,...,...
31679,31679,4471000290087200,5011000213073200,패션의류/잡화,3
31680,31680,1129000014045300,5011000319087100,가전/디지털,4
31681,31681,1129000014045300,5011000263065200,가전/디지털,6
31682,31682,4127300065073100,5011000264061200,패션의류/잡화,7


In [ ]:
df['물품_카테고리'].value_counts()

식품             25570
스포츠/레저/문화       1785
생활/건강           1374
패션의류/잡화         1113
뷰티               527
출산/유아동/반려동물      490
가전/디지털           479
가구/침구            346
Name: 물품_카테고리, dtype: int64

In [ ]:
df1 = train.copy()

In [ ]:

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df1['물품_카테고리'])

df1['물품_카테고리'] = encoder.transform(df1['물품_카테고리'])


In [ ]:
df1['물품_카테고리'].unique()

array([67, 34, 27, 17, 51, 57, 56,  5, 66,  3, 54, 80, 97, 14, 69, 15, 41,
        6, 35, 16, 81, 36, 61, 11, 44, 93, 33, 47, 29, 19, 25, 26, 45, 46,
       21, 30, 31, 77, 42, 85, 84, 71, 59, 90, 10, 70,  1, 89, 95, 75,  0,
       96, 22, 64, 86,  4, 76, 23, 74, 72, 40, 78,  9,  2, 49, 68, 73, 92,
       60, 63, 55, 50, 83, 58,  8, 99, 88, 24, 39, 38, 98, 94, 32, 87, 12,
       28, 18, 20, 13, 37, 62, 53,  7, 43, 65, 82, 48, 91, 52, 79])

In [ ]:
'''
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(train['물품_카테고리'])

train['물품_카테고리'] = encoder.transform(train['물품_카테고리'])
test['물품_카테고리'] = encoder.transform(test['물품_카테고리'])

'''

## 모델링 

In [ ]:
from lightgbm import LGBMRegressor

train_X = train.drop('운송장_건수',axis = 1)
train_Y = train['운송장_건수']

#모델 정의
model = LGBMRegressor()

In [ ]:
# 모델 학습
model.fit(train_X,train_Y)

In [ ]:
# test 데이터 예측
pred = model.predict(test)

## 정답 파일 생성